In [48]:
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
from geopy.geocoders import Nominatim
import folium
from folium import Element

In [49]:
#Reading the csv files
df_bio = pd.read_csv("tamil_nadu_biometric.csv")
df_demo = pd.read_csv("tamil_nadu_demographic.csv")
df_en = pd.read_csv("tamil_nadu_enrolment.csv")

In [50]:
#Seperating only chennai records
chennai_en=df_en[df_en['district']=='Chennai']
chennai_demo=df_demo[df_demo['district']=='Chennai']
chennai_bio=df_bio[df_bio['district']=='Chennai']

In [51]:
#Added a new col named total count, which counts total for each day
chennai_en['total_count']=chennai_en['age_0_5']+chennai_en['age_5_17']+chennai_en['age_18_greater']
chennai_bio['total_count']=chennai_bio['bio_age_5_17']+chennai_bio['bio_age_17_']
chennai_demo['total_count']=chennai_demo['demo_age_5_17']+chennai_demo['demo_age_17_']


C:\Users\User\AppData\Local\Temp\ipykernel_6576\1020554199.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chennai_en['total_count']=chennai_en['age_0_5']+chennai_en['age_5_17']+chennai_en['age_18_greater']
C:\Users\User\AppData\Local\Temp\ipykernel_6576\1020554199.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chennai_bio['total_count']=chennai_bio['bio_age_5_17']+chennai_bio['bio_age_17_']
C:\Users\User\AppData\Local\Temp\ipykernel_6576\1020554199.py:4: SettingWithCopyWarning: 
A value is trying 

In [52]:
#This here, calaculates the whole enrolments and updations for the year
print(sum(chennai_en['total_count']))
print(sum(chennai_bio['total_count']))
print(sum(chennai_demo['total_count']))

10043
233030
148368


In [53]:
# Enrolments per PIN
en_pin = chennai_en.groupby('pincode')['total_count'].sum().reset_index()
en_pin.rename(columns={'total_count':'enrolments'}, inplace=True)

# Demographic updates per PIN
demo_pin = chennai_demo.groupby('pincode')['total_count'].sum().reset_index()
demo_pin.rename(columns={'total_count':'demo_updates'}, inplace=True)

# Biometric updates per PIN
bio_pin = chennai_bio.groupby('pincode')['total_count'].sum().reset_index()
bio_pin.rename(columns={'total_count':'bio_updates'}, inplace=True)


In [54]:
from functools import reduce

dfs = [en_pin, demo_pin, bio_pin]
pin_df = reduce(lambda left, right: pd.merge(left, right, on='pincode', how='outer'), dfs)

pin_df.fillna(0, inplace=True)

pin_df['total_load'] = (
    pin_df['enrolments'] +
    pin_df['demo_updates'] +
    pin_df['bio_updates']
)

In [55]:
print(list(pin_df['pincode']))

[600001, 600002, 600003, 600004, 600005, 600006, 600007, 600008, 600009, 600010, 600011, 600012, 600013, 600014, 600015, 600016, 600017, 600018, 600019, 600020, 600021, 600022, 600023, 600024, 600025, 600026, 600028, 600029, 600030, 600031, 600032, 600033, 600034, 600035, 600036, 600037, 600038, 600039, 600040, 600041, 600042, 600049, 600050, 600051, 600053, 600057, 600058, 600060, 600061, 600066, 600068, 600077, 600078, 600079, 600080, 600081, 600082, 600083, 600084, 600085, 600086, 600087, 600088, 600089, 600090, 600091, 600092, 600093, 600094, 600095, 600096, 600097, 600099, 600100, 600101, 600102, 600103, 600104, 600106, 600107, 600108, 600110, 600112, 600113, 600114, 600115, 600116, 600117, 600118, 600119, 600125, 600126, 600129]


In [56]:
# Ensure pincode is string (no decimals)
pin_df['pincode'] = pin_df['pincode'].astype(int).astype(str)

In [57]:
T_mean = pin_df['total_load'].mean()
pin_df['load_ratio'] = pin_df['total_load'] / T_mean
pin_df['update_stress_ratio'] = (pin_df['demo_updates'] + pin_df['bio_updates']) / pin_df['total_load']

In [58]:
# Percentile-based quality scoring
q20 = pin_df['update_stress_ratio'].quantile(0.20)
q40 = pin_df['update_stress_ratio'].quantile(0.40)
q60 = pin_df['update_stress_ratio'].quantile(0.60)
q80 = pin_df['update_stress_ratio'].quantile(0.80)

def quality_star(usr):
    if usr <= q20: return 5   # best quality (low rework)
    elif usr <= q40: return 4
    elif usr <= q60: return 3
    elif usr <= q80: return 2
    else: return 1            # worst quality (high rework)

pin_df['stars'] = pin_df['update_stress_ratio'].apply(quality_star)


In [59]:
geolocator = Nominatim(user_agent="aadhaar_mapping")

def get_lat_lon(pincode):
    try:
        location = geolocator.geocode(f"{pincode}, Chennai, India")
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error for {pincode}: {e}")
    return None, None

pin_df['latitude'], pin_df['longitude'] = zip(*pin_df['pincode'].apply(get_lat_lon))


Error for 600108: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=600108%2C+Chennai%2C+India&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [60]:
m = folium.Map(location=[13.0827, 80.2707], zoom_start=11)

def get_color(load_ratio):
    if load_ratio < 0.6:
        return 'cadetblue'   # softer than gray, still underutilised
    elif load_ratio < 0.8:
        return 'green'
    elif load_ratio < 1.1:
        return 'blue'
    elif load_ratio < 1.4:
        return 'orange'
    else:
        return 'red'
           # overcrowded
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(m)

for _, row in pin_df.iterrows():
    if pd.notnull(row['latitude']):
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.Icon(
                color=get_color(row['load_ratio']),
                icon="info-sign"
            ),
            tooltip=(
                f"PIN: {row['pincode']} | "
                f"Load: {int(row['total_load'])} | "
                f"Quality: {'★'*row['stars']}{'☆'*(5-row['stars'])}"
            ),
            popup=folium.Popup(
                f"""
                <b>PIN Code:</b> {row['pincode']}<br>
                <b>Total Transactions:</b> {int(row['total_load'])}<br>
                <b>Load Ratio:</b> {row['load_ratio']:.2f}<br>
                <b>Quality:</b> {'★'*row['stars']}{'☆'*(5-row['stars'])}<br>
                <b>Update Stress:</b> {row['update_stress_ratio']:.2%}
                """,
                max_width=300
            )
        ).add_to(marker_cluster)

legend_html = '''
<div style="
position: fixed;
bottom: 40px;
left: 40px;
width: 260px;
background-color: white;
border: 2px solid grey;
z-index: 9999;
font-size: 13px;
padding: 10px;
box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
">
<b>Crowdness (Pointer Color)</b><br>
<span style="color:cadetblue;">●</span> Underutilised<br>
<span style="color:green;">●</span> Low Load<br>
<span style="color:blue;">●</span> Balanced<br>
<span style="color:orange;">●</span> High Load<br>
<span style="color:red;">●</span> Overcrowded<br><br>

<b>Data Quality (Stars in Popup)</b><br>
★★★★★ Very Good<br>
★★★☆☆ Average<br>
★☆☆☆☆ Poor
</div>
'''
m.get_root().html.add_child(Element(legend_html))
m.save("chennai_aadhaar_map.html")